In [71]:
with open('news', 'r') as f:
    text = f.read()
    news = text.split("\n\n")
    count = {'sport': 0, 'world': 0, "business": 0, "health": 0, "entertainment": 0, "sci_tech": 0,}
    for news_item in news:
        lines = news_item.split("\n")
        try:
            if lines[6] in count:
                file_to_write = open('data/' + lines[6] + '/' + str(count[lines[6]]) + '.txt', 'w+')
                count[lines[6]] = count[lines[6]] + 1
                file_to_write.write(news_item)  # python will convert \n to os.linesep
                file_to_write.close()
        except:
                pass


In [70]:
import pandas as pd
df = pd.read_json('News_Category_Dataset_v2.JSON', lines = True)
df.drop(columns=['link','authors','short_description'],axis=1,inplace=True)
df.drop(df.loc[df['category']!="CRIME"].index, inplace=True)
df = df.reset_index()
df.drop(columns='index',axis=1,inplace=True)
df.to_csv('crimetitles.csv')
crime_headlines = list(df['headline'])
count['crime'] = len(crime_headlines)
j = 1
for i in crime_headlines:     
    file_to_write = open('data/crime/' + str(j) + '.txt', 'a+')
    try:
        file_to_write.write(str(i),'\n')
        file_to_write.write('crime')
        file_to_write.close()
    except:
        pass
    j=j+1

In [72]:
count

{'sport': 8189,
 'world': 6255,
 'business': 5366,
 'health': 1851,
 'entertainment': 3286,
 'sci_tech': 2872}

In [73]:
import pandas
import glob

category_list = ["sport", "world", "business", "health", "entertainment", "sci_tech",]
directory_list = ["data/sport/*.txt", "data/world/*.txt","data/business/*.txt","data/health/*.txt","data/entertainment/*.txt","data/sci_tech/*.txt",]

text_files = list(map(lambda x: glob.glob(x), directory_list))
text_files = [item for sublist in text_files for item in sublist]

training_data = []


for t in text_files:
    f = open(t, 'r')
    f = f.read()
    t = f.split('\n')
    training_data.append({'data' : t[0] + ' ' + t[1], 'flag' : category_list.index(t[6])})
    


In [74]:
training_data = pandas.DataFrame(training_data, columns=['data', 'flag'])
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
training_data

,data,flag
0,court agrees to expedite n.f.l.'s appeal the d...,0
1,no tsunami but fifa's corruption storm rages o...,0
2,chung backs bin hammam in fifa presidential ra...,0
3,rory mcilroy hangs on to slim lead at augusta ...,0
4,bartoli beats ivanovic to reach last four at i...,0
...,...,...
27814,science and the race for the south pole in tim...,5
27815,"burglars target apple stores apple stores, ren...",5
27816,u.s.-russian crew blast off for space station ...,5
27817,google delays open access to new android softw...,5


In [75]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [76]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [116]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [123]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = ["sport", "world", "business", "health", "entertainment", "sci_tech"]

docs_new = "Actress plays cricket for the team"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(category_list[predicted[0]])






sport


In [118]:
predicted = loaded_model.predict(X_test)
result_bayes = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_bayes.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])



business  -  business
sport  -  sport
business  -  business
sci_tech  -  sci_tech
world  -  world
world  -  world
entertainment  -  entertainment
business  -  business
world  -  world
world  -  world
business  -  business
business  -  business
sport  -  health
world  -  health
health  -  health
sport  -  sport
world  -  world
world  -  world
sport  -  sport
sport  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
world  -  world
business  -  business
sport  -  entertainment
sport  -  entertainment
entertainment  -  sci_tech
business  -  business
business  -  business
business  -  business
sport  -  sport
entertainment  -  entertainment
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
business  -  business
world  -  world
sport  -  sport
world  -  world
business  -  health
sport  -  world
entertainment  -  entertainment
world  -  world
world  -  world
entertainment  -  entertainment
world  -  world
business  -  business
world  -  world
sport  - 

entertainment  -  entertainment
world  -  world
sport  -  sport
business  -  business
sport  -  sport
world  -  world
health  -  health
business  -  business
sci_tech  -  sci_tech
world  -  world
entertainment  -  entertainment
sport  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  entertainment
sci_tech  -  business
world  -  world
business  -  business
business  -  business
sci_tech  -  sci_tech
business  -  business
business  -  business
sport  -  sport
sport  -  sport
sport  -  entertainment
business  -  world
entertainment  -  entertainment
sport  -  sport
health  -  health
business  -  business
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
business  -  sci_tech
entertainment  -  entertainment
entertainment  -  entertainment
entertainment  -  entertainment
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  world
sci_tech  -  sci_tech
sport  -  sport
sport  -  s

sport  -  sport
sport  -  sport
entertainment  -  entertainment
business  -  business
world  -  world
business  -  business
sport  -  sport
world  -  world
sport  -  sport
sport  -  sport
business  -  business
world  -  world
sport  -  entertainment
sport  -  sport
sport  -  sport
world  -  world
business  -  business
entertainment  -  entertainment
sport  -  sport
health  -  health
world  -  world
business  -  health
sport  -  sport
world  -  world
world  -  sci_tech
sport  -  sport
sport  -  sport
business  -  health
sport  -  entertainment
business  -  sci_tech
sci_tech  -  sci_tech
sport  -  sport
business  -  business
world  -  world
sport  -  sport
business  -  business
business  -  business
business  -  health
sport  -  sport
business  -  health
sport  -  sport
sport  -  sport
sport  -  entertainment
business  -  business
business  -  business
world  -  world
business  -  business
entertainment  -  entertainment
world  -  world
business  -  business
sport  -  sport
sport  -  spo

sport  -  sport
world  -  world
business  -  business
sport  -  sport
business  -  sci_tech
sport  -  sport
entertainment  -  sci_tech
world  -  sci_tech
health  -  health
world  -  world
business  -  business
business  -  business
sport  -  sport
sport  -  sport
business  -  business
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
business  -  sci_tech
world  -  sci_tech
entertainment  -  entertainment
world  -  world
sport  -  sci_tech
entertainment  -  entertainment
sport  -  entertainment
sport  -  sport
sport  -  sport
entertainment  -  world
sci_tech  -  sci_tech
sport  -  sport
business  -  business
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
sport  -  sport
world  -  world
entertainment  -  health
sport  -  sport
world  -  world
world  -  health
business  -  sci_tech
business  -  business
world  -  world
world  -  world
business  -  business
business  -  business
world  -  world
sport  -  sport
sport  -  sport
world  -  world
sci_tech  -  enterta

sport  -  sport
world  -  world
business  -  business
entertainment  -  entertainment
sport  -  sport
business  -  business
world  -  world
world  -  entertainment
business  -  business
business  -  sci_tech
sport  -  sport
business  -  business
business  -  sci_tech
sci_tech  -  sci_tech
business  -  business
business  -  world
world  -  world
sport  -  sport
world  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
world  -  world
business  -  business
entertainment  -  entertainment
sport  -  sport
sport  -  sport
business  -  health
business  -  sci_tech
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  world
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
sport  -  entertainment
sport  -  sport
sport  -  sport
business  -  sci_tech
world  -  world
sport  -  sport
world  -  health
entertainment  -  entertainment
world  -  world
sport  -  sport
entertainment  -  entertainment
sport  -  sport
world  -  sci_tech
sport  -  sc

world  -  world
sport  -  sport
world  -  world
business  -  business
world  -  world
business  -  sci_tech
sport  -  sport
sport  -  sport
business  -  sci_tech
sport  -  sport
sport  -  sport
business  -  business
business  -  business
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
sport  -  health
world  -  world
sport  -  sport
sport  -  world
world  -  world
business  -  business
business  -  business
health  -  health
world  -  world
business  -  business
sport  -  sport
world  -  world
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
world  -  world
sport  -  entertainment
world  -  world
sport  -  sport
world  -  sci_tech
sport  -  sport
business  -  business
sci_tech  -  sci_tech
business  -  sci_tech
sport  -  sport
health  -  health
sport  -  sport
world  -  world
sport  -  sport
sport  -  sport
world  -  world
business  -  business
world  -  world
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
world  -  

business  -  business
sport  -  sport
world  -  health
world  -  world
entertainment  -  entertainment
entertainment  -  entertainment
world  -  entertainment
world  -  world
business  -  business
world  -  world
sport  -  sport
sci_tech  -  sci_tech
world  -  world
sport  -  entertainment
business  -  health
sport  -  sport
sport  -  health
world  -  world
sport  -  sport
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
business  -  business
sport  -  sport
sport  -  sport
world  -  world
sci_tech  -  sci_tech
sci_tech  -  sci_tech
world  -  world
entertainment  -  entertainment
sport  -  sport
entertainment  -  entertainment
business  -  sci_tech
business  -  business
world  -  world
business  -  sci_tech
business  -  business
sci_tech  -  sci_tech
world  -  world
sport  -  sport
world  -  world
world  -  world
business  -  business
sport  -  sport
sport  -  sport
business  -  business
sport  -  entertainment
business  -  business
business  -  sci_tech
business  -  business
spor

sport  -  sci_tech
sport  -  sport
world  -  world
entertainment  -  entertainment
business  -  sci_tech
world  -  world
world  -  world
sport  -  sport
sci_tech  -  sci_tech
world  -  world
sport  -  sport
business  -  health
world  -  world
entertainment  -  entertainment
sport  -  sport
business  -  business
sport  -  sci_tech
entertainment  -  entertainment
entertainment  -  entertainment
business  -  business
health  -  health
sci_tech  -  sci_tech
sport  -  sport
business  -  business
sport  -  sport
sport  -  sport
sport  -  entertainment
sport  -  sport
sci_tech  -  sci_tech
world  -  world
world  -  world
business  -  sci_tech
entertainment  -  entertainment
sport  -  sport
sport  -  sport
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
sport  -  sport
business  -  sci_tech
business  -  business
world  -  world
entert

sport  -  sport
entertainment  -  entertainment
business  -  business
sport  -  entertainment
sci_tech  -  sci_tech
sport  -  sport
business  -  business
sci_tech  -  sci_tech
sport  -  sport
world  -  world
world  -  health
world  -  world
sport  -  sport
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
world  -  world
sport  -  sport
health  -  health
business  -  business
entertainment  -  entertainment
sport  -  sport
sport  -  sport
business  -  sci_tech
sport  -  sport
sport  -  sport
business  -  business
business  -  business
business  -  sci_tech
sport  -  sport
business  -  health
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  sci_tech
sport  -  sport
sport  -  entertainment
sport  -  sport
business  -  sci_tech
world  -  world
world  -  world
world  -  world
sport  -  sport
sport  -  sport
world  -  world
business  -  business
sport  -  sport
sport  -  business
sport  -  health
sport  -  sport
business  -  health
sport  -  entertainm

sport  -  sport
sport  -  entertainment
business  -  business
entertainment  -  entertainment
sport  -  sport
business  -  business
sci_tech  -  sci_tech
sport  -  entertainment
sport  -  sport
world  -  world
business  -  business
world  -  world
business  -  world
sport  -  sport
sci_tech  -  sci_tech
business  -  sci_tech
sci_tech  -  sci_tech
business  -  business
business  -  business
business  -  business
business  -  sci_tech
business  -  business
sport  -  sport
sci_tech  -  sci_tech
world  -  world
world  -  world
sport  -  sport
business  -  business
business  -  business
sport  -  sport
business  -  business
sport  -  entertainment
entertainment  -  entertainment
business  -  sci_tech
business  -  business
world  -  world
health  -  health
business  -  business
business  -  business
entertainment  -  entertainment
world  -  world
sci_tech  -  sci_tech
business  -  sci_tech
entertainment  -  entertainment
business  -  business
sport  -  entertainment
business  -  business
bus

In [119]:
from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

[[2041   11    6    0    0    0]
 [  25 1461   60    0    6    1]
 [  69   50 1169    0   10   23]
 [  91   91  126  138    6    1]
 [ 243   40   35    0  536    4]
 [ 103   77  284    1    9  238]]


In [110]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1, max_iter= 2000)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,), max_iter=2000,
              random_state=1, solver='lbfgs')

In [111]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [112]:
predicted = clf_neural.predict(X_test)
result_softmax = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])


business  -  business
sport  -  sport
business  -  business
sci_tech  -  sci_tech
world  -  world
world  -  world
entertainment  -  entertainment
business  -  business
sci_tech  -  world
world  -  world
business  -  business
business  -  business
entertainment  -  health
health  -  health
health  -  health
sport  -  sport
world  -  world
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
world  -  world
business  -  business
entertainment  -  entertainment
entertainment  -  entertainment
business  -  sci_tech
business  -  business
business  -  business
business  -  business
sport  -  sport
entertainment  -  entertainment
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  business
world  -  world
sport  -  sport
world  -  world
health  -  health
health  -  world
entertainment  -  entertainment
world  -  world
world  -  world
entertainment  -  entertainment
world  -  world
business  - 

world  -  world
health  -  health
business  -  business
sci_tech  -  sci_tech
world  -  world
entertainment  -  entertainment
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
health  -  entertainment
sci_tech  -  business
world  -  world
business  -  business
business  -  business
sci_tech  -  sci_tech
business  -  business
business  -  business
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  world
entertainment  -  entertainment
sport  -  sport
health  -  health
entertainment  -  business
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
business  -  sci_tech
entertainment  -  entertainment
entertainment  -  entertainment
health  -  entertainment
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  world
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
entertainment  -  entertainment
sport  -  sport
entertainment  -  entertainment
busin

business  -  business
entertainment  -  entertainment
sport  -  sport
health  -  health
world  -  world
health  -  health
sport  -  sport
world  -  world
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
health  -  health
entertainment  -  entertainment
sci_tech  -  sci_tech
business  -  sci_tech
sport  -  sport
business  -  business
world  -  world
sport  -  sport
business  -  business
business  -  business
health  -  health
sport  -  sport
health  -  health
sport  -  sport
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
world  -  world
business  -  business
entertainment  -  entertainment
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
entertainment  -  entertainment
sport  -  sport
sport  -  sport
sport  -  sport
business  -  business
health  -  health
health  -  health
world  -  world
world  -  world
sport  -  sport
business  -  world
sport  -  sport
health  -  health
sport  -  spo

world  -  world
sport  -  sport
sport  -  sport
world  -  world
sci_tech  -  entertainment
world  -  world
business  -  sci_tech
sport  -  sport
business  -  business
sport  -  sport
world  -  world
sport  -  sport
health  -  health
sci_tech  -  sci_tech
world  -  world
health  -  sport
world  -  world
world  -  world
world  -  world
world  -  world
sport  -  sport
health  -  health
entertainment  -  entertainment
sport  -  sport
business  -  business
business  -  business
entertainment  -  entertainment
business  -  business
sport  -  sport
health  -  health
entertainment  -  entertainment
world  -  world
world  -  world
sport  -  sport
sport  -  sport
sci_tech  -  sci_tech
world  -  world
sci_tech  -  sci_tech
business  -  business
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
entertainment  -  entertainment
sport  -  sport
health  -  health
entertainment  -  entertainment
business  -  business
business  -  business
sport  -  sport
world  -  wo

sport  -  sport
world  -  world
business  -  business
sci_tech  -  sci_tech
business  -  business
world  -  world
sport  -  sport
world  -  world
world  -  world
world  -  business
sport  -  sport
sport  -  sport
sport  -  sport
business  -  business
business  -  business
sport  -  sport
business  -  entertainment
business  -  business
sport  -  sport
entertainment  -  entertainment
business  -  business
sport  -  sport
business  -  business
world  -  world
sport  -  sport
business  -  business
sport  -  sport
world  -  world
business  -  business
world  -  world
health  -  health
business  -  business
sci_tech  -  business
sport  -  sport
sport  -  sport
business  -  business
sci_tech  -  business
sport  -  sport
world  -  world
world  -  world
sci_tech  -  sci_tech
world  -  world
business  -  business
entertainment  -  entertainment
business  -  health
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
entertainment  -  entertainment
sci_tech  -  sci_tech
entertainment 

world  -  world
health  -  health
health  -  health
world  -  world
world  -  world
sci_tech  -  sport
sport  -  sport
business  -  business
entertainment  -  world
business  -  business
business  -  business
health  -  business
sport  -  sport
world  -  world
sport  -  sport
world  -  sport
world  -  world
health  -  health
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
business  -  business
world  -  world
health  -  entertainment
entertainment  -  entertainment
business  -  world
world  -  business
business  -  sci_tech
business  -  health
sport  -  sport
health  -  health
world  -  world
sport  -  sport
sci_tech  -  business
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
world  -  world
world  -  world
sci_tech  -  business
world  -  world
health  -  sci_tech
entertainment  -  entertainment
business  -  business
world  -  world
sci_tech  -  sci_tech
sport  -  sport
sci_tech  -  sci_tech
business  -  world
world  -  world
world  -  business
wo

business  -  business
world  -  world
sport  -  sport
world  -  world
sport  -  sport
sci_tech  -  sci_tech
sci_tech  -  business
sci_tech  -  business
business  -  business
health  -  health
business  -  business
world  -  sport
world  -  world
sci_tech  -  sci_tech
sport  -  sport
sci_tech  -  sci_tech
entertainment  -  entertainment
sport  -  sport
health  -  health
world  -  world
business  -  business
sport  -  sport
world  -  health
business  -  sci_tech
sci_tech  -  sci_tech
business  -  business
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
world  -  world
sport  -  sport
health  -  health
sport  -  sport
sport  -  sport
business  -  sci_tech
business  -  business
sci_tech  -  business
health  -  health
health  -  health
entertainment  -  entertainment
world  -  world
sport  -  sport
world  -  world
world  -  world
world  -  world
business  -  business
entertainment  -  entertainment
sport  -  s

sport  -  sport
sport  -  sport
entertainment  -  entertainment
entertainment  -  sci_tech
business  -  business
business  -  business
world  -  world
world  -  world
world  -  business
business  -  sci_tech
sci_tech  -  sport
sport  -  sport
entertainment  -  entertainment
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
sci_tech  -  health
sci_tech  -  sci_tech
entertainment  -  business
business  -  business
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
entertainment  -  entertainment
business  -  business
world  -  world
entertainment  -  entertainment
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
health  -  health
business  -  business
world  -  world
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
sport  -  sport
world  -  business
health  -  health
sport  -  sport
world  -  world
health  -  health
sport  -  sport
sport  -  sport
business  -  business
entertainment

health  -  health
sport  -  sport
sport  -  sport
world  -  business
world  -  world
world  -  world
sci_tech  -  sci_tech
business  -  business
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
world  -  world
sci_tech  -  entertainment
sport  -  sport
sport  -  sport
world  -  world
sport  -  sport
world  -  world
world  -  world
sci_tech  -  health
entertainment  -  entertainment
sci_tech  -  sci_tech
health  -  business
business  -  business
business  -  business
entertainment  -  entertainment
entertainment  -  entertainment
sport  -  sport
health  -  health
world  -  business
entertainment  -  entertainment
entertainment  -  entertainment
world  -  business
business  -  business
business  -  business
sport  -  sport
world  -  business
sport  -  sport
sport  -  sport
health  -  sci_tech
world  -  world
entertainment  -  entertainment
sport  -  sport
sport  -  sport
sport  -  sport
sci_tech  -  sci_tech
world  -  world
world  -  world
world  -  w

world  -  business
business  -  business
world  -  world
sci_tech  -  world
sci_tech  -  sci_tech
business  -  business
business  -  business
world  -  world
sport  -  sport
sci_tech  -  sci_tech
entertainment  -  entertainment
business  -  business
sport  -  sport
sport  -  sport
business  -  business
sci_tech  -  business
sport  -  sport
business  -  business
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  world
world  -  sci_tech
world  -  world
sport  -  sport
sport  -  sport
sci_tech  -  sci_tech
business  -  business
world  -  world
world  -  world
entertainment  -  entertainment
world  -  business
business  -  sci_tech
business  -  business
business  -  sci_tech
world  -  world
health  -  health
business  -  business
sport  -  sport
world  -  sci_tech
world  -  world
business  -  business
sport  -  sport
sport  -  sport
world  -  world
health  -  health
entertainment  -  entertainment
world  -  world
sci_tech  -  health
business  -  business
sport  -  s

In [113]:
from sklearn import svm
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [114]:
predicted = clf_svm.predict(X_test)
result_svm = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])

business  -  business
sport  -  sport
business  -  business
sci_tech  -  sci_tech
world  -  world
world  -  world
entertainment  -  entertainment
business  -  business
world  -  world
world  -  world
business  -  business
business  -  business
health  -  health
health  -  health
health  -  health
sport  -  sport
world  -  world
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
world  -  world
business  -  business
entertainment  -  entertainment
entertainment  -  entertainment
sci_tech  -  sci_tech
business  -  business
business  -  business
business  -  business
sport  -  sport
entertainment  -  entertainment
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
business  -  business
world  -  world
sport  -  sport
world  -  world
health  -  health
world  -  world
entertainment  -  entertainment
world  -  world
world  -  world
entertainment  -  entertainment
world  -  world
business  -  business
world 

sport  -  sport
sci_tech  -  sci_tech
entertainment  -  entertainment
world  -  world
entertainment  -  entertainment
world  -  world
entertainment  -  entertainment
world  -  world
sport  -  sport
business  -  business
sport  -  sport
world  -  world
health  -  health
business  -  business
sci_tech  -  sci_tech
world  -  world
entertainment  -  entertainment
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
entertainment  -  entertainment
sci_tech  -  business
world  -  world
business  -  business
business  -  business
sci_tech  -  sci_tech
business  -  business
business  -  business
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  world
entertainment  -  entertainment
sport  -  sport
health  -  health
business  -  business
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
business  -  sci_tech
entertainment  -  entertainment
entertainment  -  entertainment
entert

sport  -  sport
world  -  world
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
health  -  health
entertainment  -  entertainment
sci_tech  -  sci_tech
sci_tech  -  sci_tech
sport  -  sport
business  -  business
world  -  world
sport  -  sport
business  -  business
business  -  business
health  -  health
sport  -  sport
health  -  health
sport  -  sport
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
world  -  world
business  -  business
entertainment  -  entertainment
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
entertainment  -  entertainment
sport  -  sport
sport  -  sport
sport  -  sport
business  -  business
health  -  health
health  -  health
world  -  world
world  -  world
sport  -  sport
world  -  world
sport  -  sport
health  -  health
sport  -  sport
health  -  health
entertainment  -  entertainment
world  -  world
sport  -  sport
entertainment  -  entertainment
health  

sport  -  sport
world  -  world
health  -  health
world  -  world
entertainment  -  entertainment
sport  -  sport
sport  -  sport
sport  -  sport
business  -  business
world  -  world
entertainment  -  entertainment
world  -  world
health  -  health
world  -  world
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
sport  -  sport
sci_tech  -  sci_tech
world  -  world
world  -  world
entertainment  -  entertainment
sport  -  sport
sport  -  sport
world  -  world
world  -  world
business  -  business
health  -  health
world  -  world
health  -  health
world  -  world
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
sport  -  sport
entertainment  -  entertainment
sport  -  sport
business  -  business
sci_tech  -  sci_tech
sport  -  sport
business  -  business
entertainment  -  entertainment
world  -  world
world  -  world
entertainment  -  entertainment
world  -  world
world  -  world
sport  -  sport
sport  -  sport
world  -  

health  -  health
business  -  business
business  -  business
sport  -  sport
sport  -  sport
business  -  business
business  -  business
sport  -  sport
world  -  world
world  -  world
sci_tech  -  sci_tech
world  -  world
business  -  business
entertainment  -  entertainment
health  -  health
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
entertainment  -  entertainment
sci_tech  -  sci_tech
entertainment  -  entertainment
sport  -  sport
world  -  world
business  -  business
sport  -  sport
world  -  world
business  -  business
entertainment  -  entertainment
world  -  world
world  -  world
world  -  world
business  -  business
business  -  business
sport  -  sport
entertainment  -  entertainment
sport  -  sport
world  -  world
sci_tech  -  sci_tech
entertainment  -  entertainment
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
world  -  world
world  -  world
sport  -  sport
sport  -  sport
business  -  business
sport  -  sport
business  -  business


sport  -  sport
world  -  world
world  -  world
sci_tech  -  business
world  -  world
sci_tech  -  sci_tech
entertainment  -  entertainment
business  -  business
world  -  world
sci_tech  -  sci_tech
sport  -  sport
sci_tech  -  sci_tech
world  -  world
world  -  world
business  -  business
world  -  world
business  -  business
world  -  world
sport  -  sport
business  -  business
health  -  health
business  -  business
sport  -  sport
world  -  world
business  -  business
health  -  health
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
business  -  business
world  -  world
business  -  business
sport  -  sport
business  -  business
sci_tech  -  sci_tech
sci_tech  -  sci_tech
business  -  business
sport  -  sport
business  -  business
sci_tech  -  sci_tech
business  -  business
world  -  world
sci_tech  -  sci_tech
entertainment  -  entertainment
entertainment  -  entertainment
entertainment  -  entertainment
world  -  world
entertainment  -  entertainment


sport  -  sport
sci_tech  -  sci_tech
world  -  world
world  -  world
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  sport
world  -  world
sci_tech  -  sci_tech
business  -  business
sport  -  sport
world  -  world
sport  -  sport
business  -  business
sport  -  sport
sport  -  sport
world  -  world
world  -  world
sport  -  sport
world  -  world
sport  -  sport
business  -  business
world  -  world
world  -  world
world  -  world
business  -  business
business  -  business
business  -  business
business  -  business
sport  -  sport
business  -  business
entertainment  -  entertainment
health  -  health
business  -  business
business  -  business
world  -  world
sport  -  sport
business  -  business
sci_tech  -  sci_tech
sport  -  sport
business  -  business
sci_tech  -  sci_tech
business  -  business
business  -  sci_tech
world  -  world
entertainment  -  entertainment
world  -  world
world  -  world
sport  -  sport
sport  -  sport
health  -  health
health  

business  -  business
world  -  world
entertainment  -  entertainment
sport  -  sport
business  -  business
business  -  business
sport  -  sport
world  -  world
world  -  world
sport  -  sport
sport  -  sport
business  -  world
sport  -  sport
sci_tech  -  sci_tech
health  -  health
sport  -  sport
entertainment  -  health
sport  -  sport
world  -  world
entertainment  -  entertainment
entertainment  -  entertainment
entertainment  -  entertainment
world  -  world
sport  -  sport
entertainment  -  entertainment
world  -  world
business  -  business
sport  -  sport
entertainment  -  entertainment
sport  -  sport
business  -  business
sport  -  sport
sport  -  sport
world  -  world
sport  -  sport
sport  -  sport
entertainment  -  entertainment
world  -  world
sport  -  sport
sci_tech  -  sci_tech
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  entertainment
sport  -  sport
business  -  business
sport  -  sport
business  -  business
entertainment  -  entertainment
business  -  b

world  -  world
world  -  world
sci_tech  -  sci_tech
health  -  health
world  -  world
world  -  world
world  -  world
sport  -  sport
sport  -  sport
world  -  world
business  -  business
business  -  entertainment
business  -  business
sport  -  sport
health  -  health
sci_tech  -  sci_tech
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
sport  -  sport
world  -  world
health  -  health
business  -  business
world  -  world
sport  -  sport
sport  -  sport
sci_tech  -  business
health  -  health
sport  -  sport
business  -  business
sport  -  sport
business  -  business
sport  -  sport
business  -  business
sport  -  sport
entertainment  -  entertainment
business  -  business
sport  -  sport
world  -  business
business  -  business
sport  -  sport
entertainment  -  entertainment
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
sci_tech  -  sci_tech
sci_tech  -  sci_tech
world  -  wo

sci_tech  -  sci_tech
business  -  business
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
world  -  world
business  -  business
world  -  world
sport  -  sport
sci_tech  -  sci_tech
business  -  business
sci_tech  -  business
world  -  world
business  -  business
entertainment  -  entertainment
sport  -  sport
world  -  world
world  -  world
world  -  world
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
world  -  world
sport  -  sport
world  -  world
sci_tech  -  sci_tech
world  -  world
sport  -  sport
sport  -  sport
world  -  world
world  -  world
business  -  business
world  -  world
entertainment  -  entertainment
business  -  health
health  -  health
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
entertainment  -  entertainment
world  -  world
sport  -  spor

In [115]:
from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

[[2056    1    0    0    0    1]
 [   0 1533   16    3    1    0]
 [   2    5 1281    2    2   29]
 [   0    7    2  438    4    2]
 [   2    1    2    1  851    1]
 [   1    1   14    0    2  694]]
